In [74]:
import pandas as pd
import numpy as np
from WrongData import introduce_filthy_data
from sklearn.impute import KNNImputer

## Adding filthy data

In [75]:
df = pd.read_csv("./dataset/spotify_songs_restructured.csv", sep=",")
new_data = introduce_filthy_data(df, 1500)
new_data.to_csv("./dataset/spotify_songs_restructured_filthy.csv", index=False)
new_data.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,31118.000000,31209.000000,31229.000000,31199.000000,31233.000000,31220.000000,31179.000000,31183.000000,31218.000000,31162.000000,31142.000000,31216.000000,31181.000000
mean,42.435664,0.655076,0.698884,5.372576,-6.722304,0.566464,0.107144,0.174903,0.084969,0.190182,0.510686,120.865955,225873.099387
std,24.998532,0.145048,0.180837,3.612023,2.987341,0.495571,0.101376,0.219195,0.224339,0.154390,0.233022,26.885644,59770.209251
min,0.000000,0.000000,0.000175,0.000000,-46.448000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4000.000000
25%,24.000000,0.563000,0.582000,2.000000,-8.171000,0.000000,0.041000,0.015100,0.000000,0.092700,0.331000,99.957000,187800.000000
50%,45.000000,0.672000,0.721000,6.000000,-6.174000,1.000000,0.062600,0.080000,0.000016,0.127000,0.513000,121.979000,216093.000000
75%,62.000000,0.761000,0.840000,9.000000,-4.649000,1.000000,0.132000,0.254000,0.004967,0.249000,0.693000,133.881500,253613.000000
max,100.000000,0.983000,1.000000,11.000000,1.275000,1.000000,0.918000,0.994000,0.994000,0.996000,0.991000,239.440000,517810.000000


## Transforming the data

In [76]:
# nan van naam weghalen
df_clean = new_data.dropna(subset=['track_name', 'track_artist', 'track_album_name', 'track_album_release_date', 'playlist_name',
                                   'playlist_genre', 'playlist_subgenre'])
# df_clean = new_data.dropna(subset=['track_album_release_date'])
df_clean.head(10)

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.0,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718,0.930,7.0,-3.778,1.0,0.1020,0.02870,0.000009,0.2040,0.277,121.956,NaN
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,NaN,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650,0.833,1.0,-4.672,1.0,0.0359,0.08030,0.000000,0.0833,0.725,123.976,189052.0
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.0,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,pop,dance pop,0.675,0.919,8.0,-5.385,1.0,0.1270,0.07990,NaN,0.1430,0.585,124.982,163049.0
6,Never Really Over - R3HAB Remix,Katy Perry,62.0,Never Really Over (R3HAB Remix),2019-07-26,Pop Remix,pop,dance pop,0.449,0.856,NaN,-4.788,0.0,0.0623,0.18700,0.000000,0.1760,0.152,112.648,187675.0
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.0,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,Pop Remix,pop,dance pop,0.542,0.903,4.0,-2.419,0.0,0.0434,0.03350,0.000005,NaN,0.367,127.936,207619.0
8,Tough Love - Tiësto Remix / Radio Edit,Avicii,NaN,Tough Love (Tiësto Remix),2019-06-14,Pop Remix,pop,dance pop,0.594,0.935,8.0,-3.562,1.0,0.0565,0.02490,0.000004,0.6370,0.366,127.015,193187.0
10,Cross Me (feat. Chance the Rapper & PnB Rock) ...,Ed Sheeran,58.0,Cross Me (feat. Chance the Rapper & PnB Rock) ...,2019-06-21,Pop Remix,pop,dance pop,0.679,0.923,6.0,-6.500,1.0,0.1810,0.14600,0.000005,0.1240,NaN,121.984,207894.0
12,Body On My,Loud Luxury,67.0,Body On My,2019-03-29,Pop Remix,pop,dance pop,0.744,0.726,1.0,-4.675,1.0,0.0463,0.03990,0.000000,0.3740,0.687,NaN,192507.0
13,SOS - Laidback Luke Tribute Remix / Radio Edit,Avicii,68.0,SOS (Laidback Luke Tribute Remix),2019-05-17,Pop Remix,pop,dance pop,0.572,0.915,5.0,-4.451,0.0,0.0625,0.01110,0.000000,0.3390,0.678,NaN,164516.0
14,Summer Days (feat. Macklemore & Patrick Stump ...,Martin Garrix,63.0,Summer Days (feat. Macklemore & Patrick Stump ...,2019-07-12,Pop Remix,pop,dance pop,0.690,0.780,5.0,-4.446,0.0,0.0594,0.00733,0.001830,0.0729,0.238,NaN,255238.0


In [77]:
new_data.isnull().sum()

track_name                  1640
track_artist                1638
track_popularity            1715
track_album_name            1605
track_album_release_date    1647
playlist_name               1641
playlist_genre              1719
playlist_subgenre           1618
danceability                1624
energy                      1604
key                         1634
loudness                    1600
mode                        1613
speechiness                 1654
acousticness                1650
instrumentalness            1615
liveness                    1671
valence                     1691
tempo                       1617
duration_ms                 1652
dtype: int64

### Imputing missing values

In [78]:
df_clean = df_clean.fillna({
    "danceability": df_clean["danceability"].mean(), 
    "energy": df_clean.energy.mean(), 
    "key": df_clean.key.mean(), 
    "loudness": df_clean.loudness.mean(), 
    "mode": df_clean['mode'].mean(), 
    "speechiness": df_clean.speechiness.mean(), 
    "acousticness": df_clean.acousticness.mean(), 
    "instrumentalness": df_clean.instrumentalness.mean(), 
    "liveness": df_clean.liveness.mean(), 
    "valence": df_clean.valence.mean(), 
    "tempo": df_clean.tempo.mean(), 
    "duration_ms": df_clean.duration_ms.mean(),
    "track_popularity": df_clean.track_popularity.mean()
})

df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000,21337.000000
mean,42.321358,0.654992,0.699457,5.355580,-6.715209,0.564693,0.106816,0.174766,0.083254,0.191065,0.510260,120.801205,225617.442012
std,23.907969,0.139044,0.173897,3.475386,2.875234,0.476700,0.096337,0.209977,0.213584,0.149772,0.223388,25.946569,57091.048829
min,0.000000,0.078700,0.000175,0.000000,-46.448000,0.000000,0.022400,0.000001,0.000000,0.009460,0.000010,35.477000,31429.000000
25%,26.000000,0.573000,0.596000,2.000000,-7.941000,0.000000,0.042400,0.018400,0.000000,0.095400,0.346000,100.045000,190193.000000
50%,42.321358,0.656000,0.703000,5.355580,-6.431000,1.000000,0.068800,0.100000,0.000040,0.137000,0.510260,120.801205,220960.000000
75%,60.000000,0.753000,0.832000,8.000000,-4.766000,1.000000,0.121000,0.229000,0.025000,0.232000,0.676000,130.147000,249093.000000
max,100.000000,0.983000,1.000000,11.000000,0.642000,1.000000,0.877000,0.994000,0.994000,0.996000,0.991000,219.991000,516893.000000


### outliers removal

In [79]:
# Define the columns where you want to remove outliers
columns_to_clean = ["danceability", "energy", "key", "loudness", "mode", 
                    "speechiness", "acousticness", "instrumentalness", 
                    "liveness", "valence", "tempo", "duration_ms"]

# Loop through each column and apply the custom quantile-based outlier removal
for col in columns_to_clean:
    Q01 = df_clean[col].quantile(0.001)  # 0.1st percentile
    Q99 = df_clean[col].quantile(0.999)  # 99.9th percentile
    
    # Remove rows where the values in the column are outside the 1st and 99th percentile bounds
    df_clean = df_clean[(df_clean[col] >= Q01) & (df_clean[col] <= Q99)]

# Display the cleaned DataFrame
df_clean.describe()

,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
count,20933.000000,20933.000000,20933.000000,20933.000000,20933.00000,20933.000000,20933.000000,20933.000000,20933.000000,20933.000000,20933.000000,20933.000000,20933.000000
mean,42.436223,0.656330,0.700734,5.353789,-6.66453,0.563645,0.106377,0.172561,0.079204,0.190386,0.511027,120.777808,225574.810185
std,23.921808,0.137108,0.170776,3.477809,2.74101,0.476859,0.094407,0.205940,0.206768,0.147312,0.222087,25.688875,55656.410569
min,0.000000,0.176000,0.089700,0.000000,-20.38400,0.000000,0.023900,0.000006,0.000000,0.022200,0.035000,61.671000,74160.000000
25%,27.000000,0.574000,0.597000,2.000000,-7.89900,0.000000,0.042400,0.018500,0.000000,0.095500,0.347000,100.049000,190394.000000
50%,42.321358,0.657000,0.703000,5.355580,-6.40800,1.000000,0.068900,0.099400,0.000036,0.138000,0.510260,120.801205,220934.000000
75%,60.000000,0.754000,0.831000,8.000000,-4.75900,1.000000,0.121000,0.228000,0.021200,0.232000,0.676000,130.080000,248854.000000
max,100.000000,0.966000,0.995000,11.000000,-0.93600,1.000000,0.600000,0.968000,0.960000,0.967000,0.973000,204.957000,503067.000000


### Categorical Encoding

In [80]:
# Define categories and labels for track_popularity
popularity_bins = [0, 20, 40, 60, 80, 100]
popularity_labels = ['Very Low Popularity', 'Low Popularity', 'Medium Popularity', 'High Popularity', 'Very High Popularity']
df_clean['popularity_category'] = pd.Categorical(pd.cut(df_clean['track_popularity'], bins=popularity_bins, labels=popularity_labels))

# Define categories and labels for tempo
tempo_bins = [0, 60, 90, 120, 150, 180, float('inf')]
tempo_labels = ['Very Slow', 'Slow', 'Moderate', 'Fast', 'Very Fast', 'Extremely Fast']
df_clean['tempo_category'] = pd.Categorical(pd.cut(df_clean['tempo'], bins=tempo_bins, labels=tempo_labels))

# Define categories and labels for loudness
loudness_bins = [-float('inf'), -15, -10, -5, 0, float('inf')]
loudness_labels = ['Very Quiet', 'Quiet', 'Moderate', 'Loud', 'Very Loud']
df_clean['loudness_category'] = pd.Categorical(pd.cut(df_clean['loudness'], bins=loudness_bins, labels=loudness_labels))
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_name,playlist_genre,playlist_subgenre,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.000000,Call You Mine - The Remixes,2019-07-19,Pop Remix,pop,dance pop,0.718000,0.930,...,0.1020,0.028700,0.000009,0.204000,0.2770,121.956,225617.442012,Medium Popularity,Fast,Loud
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,42.321358,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,pop,dance pop,0.650000,0.833,...,0.0359,0.080300,0.000000,0.083300,0.7250,123.976,189052.000000,Medium Popularity,Fast,Loud
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.000000,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,Pop Remix,pop,dance pop,0.675000,0.919,...,0.1270,0.079900,0.083254,0.143000,0.5850,124.982,163049.000000,High Popularity,Fast,Moderate
6,Never Really Over - R3HAB Remix,Katy Perry,62.000000,Never Really Over (R3HAB Remix),2019-07-26,Pop Remix,pop,dance pop,0.449000,0.856,...,0.0623,0.187000,0.000000,0.176000,0.1520,112.648,187675.000000,High Popularity,Moderate,Loud
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.000000,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,Pop Remix,pop,dance pop,0.542000,0.903,...,0.0434,0.033500,0.000005,0.191065,0.3670,127.936,207619.000000,High Popularity,Fast,Loud
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.000000,City Of Lights (Vocal Mix),2014-04-28,♥ EDM LOVE 2020,edm,progressive electro house,0.654992,0.922,...,0.0936,0.076600,0.000000,0.066800,0.2100,128.170,204375.000000,Medium Popularity,Fast,Loud
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.000000,Closer Remixed,2013-03-08,♥ EDM LOVE 2020,edm,progressive electro house,0.522000,0.786,...,0.0420,0.001710,0.004270,0.375000,0.4000,128.041,225617.442012,Very Low Popularity,Fast,Loud
32830,Sweet Surrender - Radio Edit,Starkillers,14.000000,Sweet Surrender (Radio Edit),2014-04-21,♥ EDM LOVE 2020,edm,progressive electro house,0.529000,0.821,...,0.0481,0.108000,0.000001,0.150000,0.4360,127.989,210112.000000,Very Low Popularity,Fast,Loud
32831,Only For You - Maor Levi Remix,Mat Zo,15.000000,Only For You (Remixes),2014-01-01,♥ EDM LOVE 2020,edm,progressive electro house,0.626000,0.888,...,0.1090,0.007920,0.127000,0.343000,0.3080,128.008,367432.000000,Very Low Popularity,Fast,Moderate


## Feature splitting for release_date (Domain expertise)

In [81]:
# Convert track_album_release_date to datetime, coercing errors
df_clean['track_album_release_date'] = pd.to_datetime(df_clean['track_album_release_date'], errors='coerce')

# Extract the month name and create a new column
df_clean['release_month'] = df_clean['track_album_release_date'].dt.strftime('%B')

# Display the DataFrame
df_clean.drop(columns=['instrumentalness', 'acousticness', 'mode', 'playlist_subgenre', 'playlist_name'], inplace=True)
df_clean

,track_name,track_artist,track_popularity,track_album_name,track_album_release_date,playlist_genre,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms,popularity_category,tempo_category,loudness_category,release_month
3,Call You Mine - Keanu Silva Remix,The Chainsmokers,60.000000,Call You Mine - The Remixes,2019-07-19,pop,0.718000,0.930,7.00000,-3.778000,0.1020,0.204000,0.2770,121.956,225617.442012,Medium Popularity,Fast,Loud,July
4,Someone You Loved - Future Humans Remix,Lewis Capaldi,42.321358,Someone You Loved (Future Humans Remix),2019-03-05,pop,0.650000,0.833,1.00000,-4.672000,0.0359,0.083300,0.7250,123.976,189052.000000,Medium Popularity,Fast,Loud,March
5,Beautiful People (feat. Khalid) - Jack Wins Remix,Ed Sheeran,67.000000,Beautiful People (feat. Khalid) [Jack Wins Remix],2019-07-11,pop,0.675000,0.919,8.00000,-5.385000,0.1270,0.143000,0.5850,124.982,163049.000000,High Popularity,Fast,Moderate,July
6,Never Really Over - R3HAB Remix,Katy Perry,62.000000,Never Really Over (R3HAB Remix),2019-07-26,pop,0.449000,0.856,5.35558,-4.788000,0.0623,0.176000,0.1520,112.648,187675.000000,High Popularity,Moderate,Loud,July
7,Post Malone (feat. RANI) - GATTÜSO Remix,Sam Feldt,69.000000,Post Malone (feat. RANI) [GATTÜSO Remix],2019-08-29,pop,0.542000,0.903,4.00000,-2.419000,0.0434,0.191065,0.3670,127.936,207619.000000,High Popularity,Fast,Loud,August
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32828,City Of Lights - Official Radio Edit,Lush & Simon,42.000000,City Of Lights (Vocal Mix),2014-04-28,edm,0.654992,0.922,2.00000,-1.814000,0.0936,0.066800,0.2100,128.170,204375.000000,Medium Popularity,Fast,Loud,April
32829,Closer - Sultan & Ned Shepard Remix,Tegan and Sara,20.000000,Closer Remixed,2013-03-08,edm,0.522000,0.786,0.00000,-4.462000,0.0420,0.375000,0.4000,128.041,225617.442012,Very Low Popularity,Fast,Loud,March
32830,Sweet Surrender - Radio Edit,Starkillers,14.000000,Sweet Surrender (Radio Edit),2014-04-21,edm,0.529000,0.821,5.35558,-4.899000,0.0481,0.150000,0.4360,127.989,210112.000000,Very Low Popularity,Fast,Loud,April
32831,Only For You - Maor Levi Remix,Mat Zo,15.000000,Only For You (Remixes),2014-01-01,edm,0.626000,0.888,2.00000,-6.715209,0.1090,0.343000,0.3080,128.008,367432.000000,Very Low Popularity,Fast,Moderate,January


In [82]:
df_clean['track_album_release_date'] = df_clean['track_album_release_date'].fillna(0)
df_clean = df_clean[df_clean['track_album_release_date'] != '0']
df_clean.describe()


,track_popularity,danceability,energy,key,loudness,speechiness,liveness,valence,tempo,duration_ms
count,20933.000000,20933.000000,20933.000000,20933.000000,20933.00000,20933.000000,20933.000000,20933.000000,20933.000000,20933.000000
mean,42.436223,0.656330,0.700734,5.353789,-6.66453,0.106377,0.190386,0.511027,120.777808,225574.810185
std,23.921808,0.137108,0.170776,3.477809,2.74101,0.094407,0.147312,0.222087,25.688875,55656.410569
min,0.000000,0.176000,0.089700,0.000000,-20.38400,0.023900,0.022200,0.035000,61.671000,74160.000000
25%,27.000000,0.574000,0.597000,2.000000,-7.89900,0.042400,0.095500,0.347000,100.049000,190394.000000
50%,42.321358,0.657000,0.703000,5.355580,-6.40800,0.068900,0.138000,0.510260,120.801205,220934.000000
75%,60.000000,0.754000,0.831000,8.000000,-4.75900,0.121000,0.232000,0.676000,130.080000,248854.000000
max,100.000000,0.966000,0.995000,11.000000,-0.93600,0.600000,0.967000,0.973000,204.957000,503067.000000


In [83]:
df_clean.to_csv("./dataset/spotify_songs_restructured_cleaned.csv", index=False)